## Summary
Notebook to translate the mode and purpose into english
get script

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely

%matplotlib inline

In [2]:
gdf = gpd.read_file("../app_route_data/trajets_mtl_trajet_2017.shp", encoding='utf-8') # utf-8 needed to read french letters

In [3]:
gdf.head()

,id_trip,mode,purpose,starttime,endtime,journeytim,distance_m,geometry
0,1547,None,None,2017-09-18 04:16:58UTC,2017-09-18 04:26:26UTC,568,1530.222174,LINESTRING (7624015.797731051 1247372.35912242...
1,308312,None,None,2017-09-18 06:17:46UTC,2017-09-18 06:58:49UTC,2463,324.869460,LINESTRING (7624029.565886391 1247375.82561127...
2,384772,None,None,2017-09-18 09:30:24UTC,2017-09-18 10:17:28UTC,2824,32009.701647,(LINESTRING (7607307.10774919 1253237.70694762...
3,150744,car,pick_up_a_person,2017-09-18 10:02:50UTC,2017-09-18 10:17:12UTC,862,9935.922336,LINESTRING (7624322.265039731 1247672.97005955...
4,199011,None,None,2017-09-18 10:18:40UTC,2017-09-18 10:27:44UTC,544,563.806088,(LINESTRING (7616402.39358294 1246769.19969164...


In [4]:
purpose_translations = {"None":"None",'Reconduire / aller chercher une personne':'pick_up_a_person',
       "Travail / Rendez-vous d'affaires":'work', 'Magasinage / emplettes':'shops',
       'Retourner à mon domicile':'returning_home', 'Santé':'health', 'Loisir':'leisure', 'Éducation':'education',
       'Autre':'other', 'Repas / collation / café':'cafe', 'ND':'not_available'}
if not "other" in gdf.purpose.unique():
    gdf['purpose'] = gdf.purpose.apply(lambda pur: purpose_translations[pur] if pur else pur)
else:
    print("passed")

In [5]:
mode_translations = {"Voiture / Moto":"car", "À pied":"walking", "Transport collectif":"public_transport",
                     "Autopartage":"car_sharing", "Vélo":"cycling", "Taxi":"taxi",
                     "Autre":"other", "ND":"not_available"}
def mode_to_new_name(mode_names):
    if mode_names:
        mode_list = mode_names.split(',')
        translations = []
        for mode in mode_list:
            translations.append(mode_translations[mode.lstrip()])
        return str(translations).strip('[]').replace("'", '')
    else:
        return mode_names

In [6]:
if not "other" in gdf['mode'].unique():
    gdf['mode'] = gdf['mode'].apply(mode_to_new_name)

In [7]:
gdf['mode'].unique()

array([None, 'car', 'walking, public_transport', 'public_transport',
       'walking', 'cycling', 'walking, public_transport, cycling',
       'public_transport, car', 'walking, cycling',
       'walking, car, cycling', 'car_sharing', 'other',
       'public_transport, cycling', 'walking, public_transport, car',
       'public_transport, car, cycling', 'walking, car', 'car, cycling',
       'taxi', 'not_available', 'taxi, cycling',
       'walking, car_sharing, cycling',
       'car_sharing, public_transport, car', 'taxi, car',
       'walking, car_sharing, car', 'car_sharing, other',
       'walking, taxi, cycling', 'car_sharing, taxi', 'car_sharing, car',
       'car_sharing, other, car', 'car_sharing, public_transport',
       'walking, public_transport, car, cycling',
       'other, public_transport', 'walking, other, public_transport',
       'other, cycling', 'other, taxi',
       'walking, car_sharing, other, public_transport', 'walking, other',
       'other, car', 'walking, ta

In [125]:
## re-save the file
# gdf.to_file("../app_route_data/trajets_mtl_trajet_2017.shp", encoding='utf-8')

## Translate supplementary data

In [8]:
mtl_roads = gpd.read_file('../data/mtl_roads.json')
mtl_street_parking = gpd.read_file('../data/signalisation_stationnement.geojson')

In [9]:
mtl_land_use = gpd.read_file('../data/mtl_land_use.json')
mtl_culture_sites = gpd.read_file('../data/lieuxculturels.json')

### MTL ROADS
CLASS of roads ->
-	Class 9 - Projected Street 
-	class 8 - Highways 
-	class 7 - Main arteries 
-	Class 6 - Secondary arteries 
-	Class 5 - Collectors 
-	class 4 - Private 
-	class 3 - Wharf 
-	class 2 - Business Places 
-	Class 1 - Certain pedestrian routes 
-	Class 0 - Local Streets 

Circulation direction (CIRC_DIRECTION)
-	1 -> one way in the direction of digitization 
-	-1 -> one way in the opposite direction of digitization 
-	0 -> both ways 

In [11]:
mtl_roads.rename(columns={"CLASSE":"CLASS","TYP_VOIE":"PATH_TYPE","PATH_NAME":"NOM_VOIE",
                          "DIR_VOIE":"DIRECTION", "SENS_CIR":"CIRC_DIRECTION"}, inplace=True)

In [13]:
## note chemin and voie are grouped into path and place and carré into square
path_type_translations = {"place":"square","autoroute":"highway","chemin":"path","voie":"path",\
                         "croissant":"crescent","ruelle":"alley", "allée":"alley", "côte":"coast","cercle":"roundabout",\
                          "pont":"bridge","crête":"crest/ridge","route":"road","montée":"climb",\
                         "carré":"square","cours":"course","carrefour":"crossroads", "pont-tunnel":"bridge-tunnel"}
if "pont" in mtl_roads.PATH_TYPE.unique():
    print("making translations")
    mtl_roads['PATH_TYPE'] = mtl_roads.PATH_TYPE.apply(lambda path: path_type_translations[path] if path in path_type_translations.keys() else path)
    
mtl_roads.PATH_TYPE.unique()

array(['avenue', 'square', 'highway', 'rue', 'path', 'boulevard',
       'terrasse', 'crescent', 'allée', 'alley', 'coast', 'roundabout',
       None, 'promenade', 'bridge', 'jardin', 'crest/ridge', 'parc',
       'passage', 'road', 'climb', 'impasse', 'circuit', 'rond-point',
       'course', 'Lane', 'tunnel', 'crossroads', 'bridge-tunnel'],
      dtype=object)

## MTL Street Parking

In [14]:
# select only real stations
mtl_street_parking = mtl_street_parking.loc[(mtl_street_parking.DESCRIPTION_REP == "Réel") & (mtl_street_parking.DESCRIPTION_CAT == "STATIONNEMENT")] 

In [15]:
mtl_street_parking.head()

,POTEAU_ID_POT,POSITION_POP,PANNEAU_ID_PAN,PANNEAU_ID_RPA,DESCRIPTION_RPA,CODE_RPA,FLECHE_PAN,DESCRIPTION_CAT,POTEAU_VERSION_POT,DATE_CONCEPTION_POT,DESCRIPTION_REP,DESCRIPTION_RTP,X,Y,Longitude,Latitude,NOM_ARROND,PAS_SUR_RUE,TOPONYME_PAN,geometry
3,1336,6,2461,2585,\P 09h30-10h30 MAR. 1 MARS AU 1 DEC.,SU-MD,0,STATIONNEMENT,0,1999-04-21,Réel,4- Poteau en bois,298028.806,5047372.523,-73.586745,45.566230,Villeray - Saint-Michel - Parc-Extension,NaN,None,POINT (7628218.145379757 1251167.295888311)
7,4250,7,7788,2452,\P LIVRAISON SEULEMENT 09h30-18h LUN. AU VEN.,SL-JS,3,STATIONNEMENT,1,NaT,Réel,1- Tige et base,301779.614,5044836.449,-73.538678,45.543436,Mercier - Hochelaga-Maisonneuve,NaN,None,POINT (7632599.829611734 1249783.112941834)
8,4298,7,7945,2413,\P EN TOUT TEMPS,SD-TT,0,STATIONNEMENT,0,1999-03-05,Réel,4- Poteau en bois,300482.236,5044646.690,-73.555290,45.541721,Mercier - Hochelaga-Maisonneuve,NaN,None,POINT (7631392.336576609 1249223.714446473)
15,4495,7,8336,11,\A EN TOUT TEMPS,AD-TT,0,STATIONNEMENT,0,1999-03-05,Réel,3- Fût,295707.531,5046220.492,-73.616461,45.555837,Villeray - Saint-Michel - Parc-Extension,NaN,None,POINT (7626292.248371616 1249376.364656074)
16,4499,7,8346,2539,\P 09h30-10h30 JEU. 1 MARS AU 1 DEC.,SU-JD,0,STATIONNEMENT,0,1999-03-05,Réel,4- Poteau en bois,296705.034,5046186.599,-73.603684,45.555544,Villeray - Saint-Michel - Parc-Extension,NaN,None,POINT (7627272.570033573 1249631.315670016)


## MTL Land use

In [16]:
category_translations = {'mixte':"mixed", 'residentiel':"residential", 'activites diversifiees':"diversified activities",\
                         'emplois':"employment", 'institution':"institution", 'conservation':"conservation",\
                         'religieux':"religious", 'agricole':"agricultural", 'parc':"park",\
                         'infrastructure':"infrastructure"}

if "mixte" in mtl_land_use.categorie.unique():
    print("making translations")
    mtl_land_use['categorie'] = mtl_land_use.categorie.apply(lambda lu: category_translations[lu] if lu in category_translations.keys() else lu)

## MTL culture sites

In [18]:
mtl_culture_sites.rename(columns={"Noms du réseau":"type","Nom du lieu culturel municipal":"name"}, inplace=True)

In [19]:
culture_site_translations = {'Espace pour la vie,Musée municipal':"Space for Life, Municipal Museum",\
                             'Bibliothèque':'Library', 'Salle de spectacle':'Auditorium',\
                             'Salle de spectacle,Musée municipal':'Auditorium, Municipal Museum',
                             'Église':'Church', 'Centre des loisirs':'Recreation Center',\
                             'Centre communautaire':'Community Center', 'Cinéma':'Cinema',\
                             'Bibliothèque,Centre communautaire,Salle de spectacle': 'Library, Community Center, Theater',\
                             'Théâtre':'Theater', 'Cinéma,Théâtre':'Cinema, Theater', 'Musée municipal':'Municipal Museum'
                            }
if "Église" in mtl_culture_sites['type'].unique():
    print("making translations")
    mtl_culture_sites['type'] = mtl_culture_sites['type'].apply(lambda lu: culture_site_translations[lu] if lu in culture_site_translations.keys() else lu)
mtl_culture_sites['type'].unique()

array(['Space for Life, Municipal Museum', 'Library', 'Auditorium',
       'Auditorium, Municipal Museum', 'Church', 'Recreation Center',
       'Community Center', 'Cinema', 'Library, Community Center, Theater',
       'Theater', 'Cinema, Theater', 'Municipal Museum'], dtype=object)

### re-save files

In [21]:
# %%time
mtl_culture_sites.to_file('../data/lieuxculturels.json', driver="GeoJSON")
mtl_roads.to_file('../data/mtl_roads.json', driver="GeoJSON")
mtl_street_parking.to_file('../data/signalisation_stationnement.geojson', driver="GeoJSON")


In [22]:
# %%time
mtl_land_use.to_file('../data/mtl_land_use.json', driver="GeoJSON")